<a href="https://colab.research.google.com/github/rafaelwrossi/alura_desafio/blob/main/anjo_da_guarda_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q -U google-generativeai
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display, Markdown
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib.pyplot as plt
import requests
import os

from google.colab import userdata
api_key = userdata.get("chave_secreta")
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Dados históricos de enchentes
dados_historicos = {
    "Porto Alegre": [
        {"data": "15/07/2010", "descricao": "Enchente leve."},
        {"data": "20/09/2011", "descricao": "Enchente moderada."},
        {"data": "12/08/2012", "descricao": "Grande enchente, muitos danos."},
        {"data": "17/07/2013", "descricao": "Enchente leve."},
        {"data": "20/09/2014", "descricao": "Enchente moderada."},
        {"data": "15/06/2015", "descricao": "Grande enchente, áreas baixas muito afetadas."},
        {"data": "25/08/2016", "descricao": "Enchente moderada."},
        {"data": "30/07/2017", "descricao": "Enchente leve."},
        {"data": "20/09/2018", "descricao": "Enchente moderada, com prejuízos localizados."},
        {"data": "15/10/2019", "descricao": "Enchente severa, pior da década."}
    ],
    "Curitiba": [
        {"data": "10/06/2010", "descricao": "Enchente rápida após chuva intensa."},
        {"data": "03/05/2011", "descricao": "Enchente leve."},
        {"data": "20/09/2012", "descricao": "Enchente moderada."},
        {"data": "17/07/2013", "descricao": "Grande enchente, danos extensivos."},
        {"data": "20/09/2014", "descricao": "Enchente leve."},
        {"data": "15/06/2015", "descricao": "Enchente moderada."},
        {"data": "25/08/2016", "descricao": "Enchente leve."},
        {"data": "30/07/2017", "descricao": "Enchente severa."},
        {"data": "20/09/2018", "descricao": "Chuvas fortes causaram inundações em vários bairros."},
        {"data": "15/10/2019", "descricao": "Enchente moderada."}
    ],
    "Florianópolis": [
        {"data": "22/10/2010", "descricao": "Enchente leve."},
        {"data": "18/11/2011", "descricao": "Enchente moderada."},
        {"data": "14/10/2012", "descricao": "Enchente severa."},
        {"data": "19/09/2013", "descricao": "Enchente leve."},
        {"data": "20/10/2014", "descricao": "Enchente leve, mas com alerta de deslizamento."},
        {"data": "22/10/2015", "descricao": "Enchente leve."},
        {"data": "30/11/2016", "descricao": "Enchente moderada."},
        {"data": "25/09/2017", "descricao": "Enchente severa, alerta de deslizamento."},
        {"data": "27/10/2018", "descricao": "Enchente leve."},
        {"data": "30/09/2019", "descricao": "Enchente moderada."}
    ]
}

areas_de_risco = {
    "Porto Alegre": ["Centro", "Bairro X"],
    "Curitiba": ["Bairro Y", "Zona Z"],
    "Florianópolis": ["Bairro J", "Zona W"]
}

# Interface interativa
dropdown_cidades = widgets.Dropdown(
    options=['Selecione uma cidade'] + list(dados_historicos.keys()),
    value='Selecione uma cidade',
    description='Cidade:',
    disabled=False,
)

output = widgets.Output()

def display_markdown(text):
    display(Markdown(text))

def ao_selecionar_cidade(change):
    with output:
        clear_output(wait=True)
        if change['new'] != 'Selecione uma cidade':
            alerta = gerar_alerta(change['new'])
            display_markdown(alerta)
            plotar_dados_historicos(change['new'])

dropdown_cidades.observe(ao_selecionar_cidade, names='value')
display(dropdown_cidades, output)

def obter_previsao_tempo(cidade):
    api_key = userdata.get('openwheater')
    genai.configure(api_key=api_key)
    url = f"http://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={api_key}&units=metric&lang=pt"
    response = requests.get(url)
    if response.status_code == 200:
        dados_tempo = response.json()
        temperatura = dados_tempo['main']['temp']
        descricao = dados_tempo['weather'][0]['description']
        umidade = dados_tempo['main']['humidity']
        vento = dados_tempo['wind']['speed']
        return f"Temperatura: {temperatura}°C, Condição: {descricao}, Umidade: {umidade}%, Vento: {vento} km/h"
    else:
        return f"Erro ao obter previsão do tempo: {response.status_code}"

def gerar_alerta(localizacao):
    historico = dados_historicos.get(localizacao, [])
    historico_formatado = ', '.join([f"{evento['data']} ({evento['descricao']})" for evento in historico])
    risco = "alto" if localizacao in areas_de_risco else "moderado"
    previsao_tempo = obter_previsao_tempo(localizacao)
    alerta = f"Alerta de enchente em {localizacao}.\nHistórico: {historico_formatado}\nRisco: {risco}\nPrevisão do Tempo: {previsao_tempo}"
    return alerta

def plotar_dados_historicos(localizacao):
    datas = [evento['data'] for evento in dados_historicos[localizacao]]
    descricoes = [evento['descricao'] for evento in dados_historicos[localizacao]]
    plt.figure(figsize=(10, 5))
    plt.plot(datas, descricoes, marker='o')
    plt.title(f"Histórico de Enchentes em {localizacao}")
    plt.xlabel("Data")
    plt.ylabel("Descrição")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()


Dropdown(description='Cidade:', options=('Selecione uma cidade', 'Porto Alegre', 'Curitiba', 'Florianópolis'),…

Output()